In [1]:
import sys

sys.path.append("../")

import os

import hydra
import numpy as np
import torch
import torch.nn.functional as F
import wandb
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig
from termcolor import cprint
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from tqdm import tqdm

from lib.datasets import ThingsMEGDataset
from lib.models import BasicConvClassifier
from lib.utils import set_seed

data_dir = "../../data"
batch_size = 32
num_workers = 1
loader_args = {"batch_size": batch_size, "num_workers": num_workers}


In [2]:
from lib.datasets import ThingsMEGDatasetWithImages
train_set_with_images = ThingsMEGDatasetWithImages("train", data_dir)
train_loader_with_images = DataLoader(train_set_with_images, shuffle=False, **loader_args)

In [3]:
for image_X, brain_X, y, subject_idx in train_loader_with_images:
    print(image_X.shape)
    print(brain_X.shape)
    print(y.shape)
    print(subject_idx.shape)
    break

    

torch.Size([32, 3, 224, 224])
torch.Size([32, 271, 281])
torch.Size([32])
torch.Size([32])


In [4]:
image_module = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")
print(image_module)

Using cache found in /Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main
/Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/hayato/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [6]:
train_set_with_images.save_embedded_images(image_module, model_id="facebookresearch/dinov2/dinov2_vits14")

TypeError: ThingsMEGDatasetWithImages.save_embedded_images() missing 1 required positional argument: 'model_id'